# Price Predictor Parameter Optimisation

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Literal

from prompt_toolkit.shortcuts import set_title
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

In [2]:
ROOT = Path().resolve().parent

DATA = ROOT / "data"
DAILY_PRICES = DATA / "daily_price_downloads"
MINUTELY_PRICES = DATA / "minutely_price_downloads"
INTERPOLATED_PRICES = DATA / "interpolated_prices"

SAVED_MODELS = DATA / "saved_models"

## Class Setup

In [3]:
class LSTMPredictor:
    """ LSTM based stock price predictor framework. """
    def __init__(self,
                 sampling_rate_minutes: int,  # required
                 price_csv_path: str = None,  # data import parameters
                 price_column: str = 'close',
                 date_column: str = 'date',
                 daily_prediction_hour: int = None,  # data preparation parameters
                 rolling_window_size: int = 32,
                 forecast_horizon: int = 12,
                 validation_split: float = 0.2,
                 batch_size: int = 32,
                 hidden_lstm_layer_size: int = 64,  # model parameters
                 n_lstm_layers: int = 4,
                 dropout: float = 0.2,
                 use_final_hidden_state: bool = False,
                 use_pre_lstm_fc_layer: bool = True,
                 init_weights: bool = True,
                 use_mps_if_available: bool = False,
                 forecast_step_loss_weight_range: (float) = (1, 0.4),  # training parameters
                 n_train_epochs: int = None,  # if set to some number, trains upon initialisation
                 lr_scheduler: Literal['step', 'plateau'] = 'plateau',
                 initial_lr: float = 0.001,
                 step_scheduler_step_size: int = 40,
                 plateau_scheduler_factor: float = 0.5,
                 early_stopping_patience: int = 30,  # if 0 doesn't utilise early stopping
                 ):
        self._sampling_rate_minutes = sampling_rate_minutes
        self._price_csv_path = price_csv_path
        if price_csv_path is None: print('No price file provided yet. Define LSTMPredictor.price_csv_path to trigger data import.')
        self._price_column = price_column
        self._date_column = date_column
        self._daily_prediction_hour = daily_prediction_hour
        if daily_prediction_hour is None: print('No daily prediction hour defined yet, hence currently predictions are carried out at every time step. Define LSTMPredictor.daily_prediction_hour to change this.')
        self._rolling_window_size = rolling_window_size
        self._forecast_horizon = forecast_horizon
        self._validation_split = validation_split
        self._batch_size = batch_size
        self._hidden_lstm_layer_size = hidden_lstm_layer_size
        self._n_lstm_layers = n_lstm_layers
        self._dropout = dropout
        self._use_final_hidden_state = use_final_hidden_state
        self._use_pre_lstm_fc_layer = use_pre_lstm_fc_layer
        self._init_weights = init_weights
        self._use_mps_if_available = use_mps_if_available
        self._forecast_step_loss_weight_range = forecast_step_loss_weight_range
        self._n_train_epochs = n_train_epochs
        if n_train_epochs is None: print('No training epochs defined upon initialisation. Define LSTMPredictor.n_train_epochs to start training procedure.')
        self._lr_scheduler = lr_scheduler
        self._initial_lr = initial_lr
        self._step_scheduler_step_size = step_scheduler_step_size
        self._plateau_scheduler_factor = plateau_scheduler_factor
        self._early_stopping_patience = early_stopping_patience

    @property
    def sampling_rate_minutes(self): return self._sampling_rate_minutes

    @property
    def price_csv_path(self): return self._price_csv_path
    
    @price_csv_path.setter
    def price_csv_path(self, value): self._price_csv_path = value; self.import_data()
    
    @property
    def price_column(self): return self._price_column

    @property
    def date_column(self): return self._date_column

    @property
    def daily_prediction_hour(self): return self._daily_prediction_hour

    @daily_prediction_hour.setter
    def daily_prediction_hour(self, value): self._daily_prediction_hour = value; self.prepare_data()
    
    @property
    def rolling_window_size(self): return self._rolling_window_size

    @property
    def forecast_horizon(self): return self._forecast_horizon

    @property
    def validation_split(self): return self._validation_split

    @property
    def batch_size(self): return self._batch_size

    @property
    def hidden_lstm_layer_size(self): return self._hidden_lstm_layer_size

    @property
    def n_lstm_layers(self): return self._n_lstm_layers

    @property
    def dropout(self): return self._dropout

    @property
    def use_final_hidden_state(self): return self._use_final_hidden_state

    @property
    def use_pre_lstm_fc_layer(self): return self._use_pre_lstm_fc_layer

    @property
    def init_weights(self): return self._init_weights

    @property
    def use_mps_if_available(self): return self._use_mps_if_available

    @property
    def forecast_step_loss_weight_range(self): return self._forecast_step_loss_weight_range

    @property
    def n_train_epochs(self): return self._n_train_epochs

    @n_train_epochs.setter
    def n_train_epochs(self, value): self._n_train_epochs = value; self.run_training()
    
    @property
    def lr_scheduler(self): return self._lr_scheduler

    @property
    def initial_lr(self): return self._initial_lr

    @property
    def step_scheduler_step_size(self): return self._step_scheduler_step_size

    @property
    def plateau_scheduler_factor(self): return self._plateau_scheduler_factor

    @property
    def early_stopping_patience(self): return self._early_stopping_patience
        
    def import_data(self):
        pass
    
    def prepare_data(self):
        pass
    
    def run_training(self):
        pass